# Test CQGC-utils/lib

Use this notebook to try out code for the development of packages under "CQGC-utils/lib".

In [1]:
import os, sys
import time
import json
import pandas as pd

src_path = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(src_path)
from lib.nanuq import Nanuq
from lib.gapp import Phenotips


In [2]:
def now():
    return(time.strftime('[%Y-%m-%d@%H:%M:%S]'))


def format_mrn_eid(ep, mrn):
    if ep == 'CHUSJ':
        mrn = mrn.lstrip('0')
    elif ep == 'CHUS':
        pass
    return(ep + mrn)


def get_fastq_paths_on_bs(biosample):
    paths = []
    return(paths)


In [16]:
nq = Nanuq()
samplenames = nq.get_samplenames("A00516_0426")
if not samplenames.text.startswith("##20"):
    sys.exit(f"{now()} ERROR: Unexpected content for SampleNames. Please verify Nanuq's reponse:\n{samplenames.text}")
else:
    print(f"{now()} Retrieved samples conversion table from Nanuq")

pho   = Phenotips() # Interact with Phenotips REST API


[2023-07-27@11:59:32] Connecting to https://nanuq.cqgc.hsj.rtss.qc.ca/nanuqMPS/sampleConversionTable/run/A00516_0426/technology/NovaSeq/
[2023-07-27@11:59:33] Retrieved samples conversion table from Nanuq


In [25]:
familyId2pid = {}    # Lookup table {'suname': 'pid', 'suname': 'pid',...}

# Get Nanuq JSON for each CQGC ID found in the SampleNames (returned
# as a string by requests.text) and parse sample infos. 
# SampleNames lines are tab-delimitted. Comment lines begin with "#".
# Results are stored in `cases` and printed to STDOUT at the end.
# 
cases = []          # Case records
for line in samplenames.text.splitlines():
    if not line.startswith('#'):
        cqgc, sample = line.split("\t")
        data = json.loads(nq.get_sample(cqgc))
        print(f"{now()} Got information for biosample {cqgc} a.k.a. {sample}")

        if len(data) != 1:
            print(f"{now()} WARNING: Number of samples retrieved from Nanuq is not 1.\n{data}")

        sample_infos = [
            data[0]["ldmSampleId"],
            data[0]["labAliquotId"],
            data[0]["patient"]["familyMember"],
            data[0]["patient"]["sex"],
            data[0]["patient"]["ep"],
            data[0]["patient"]["mrn"],
            data[0]["patient"]["designFamily"],
            data[0]["patient"]["birthDate"],
            data[0]["patient"]["status"],
            data[0]["patient"].get("familyId", "-")
        ]

        # Add Phenotips ID (`pid`) and patients' HPO identifiers
        # Lookup this information in Phenotips, using the EP+MRN
        # Ex: CHUSJ123456
        #
        ep_mrn = format_mrn_eid(data[0]["patient"]["ep"], data[0]["patient"]["mrn"])
        print(f"{now()} Getting HPO terms from Phenotips by Labeled EID {ep_mrn}\n")

        patient   = pho.get_patient_by_mrn(ep_mrn)
        hpo_ids   = []
        hpo_labels= []
        if patient is not None:
            pid = patient['id']
            hpos = pho.parse_hpo(patient)
            for hpo in hpos:
                hpo_ids.append(hpo['id'])
                hpo_labels.append(hpo['label'])
        else:
            pid = ''

        if len(hpo_ids) == 0:
            ids_str = ''
            labels_str= ''
        else:
            ids_str = ','.join(hpo_ids)
            labels_str = ','.join(hpo_labels)

        sample_infos.append(pid)
        sample_infos.append(labels_str)
        sample_infos.append(ids_str)

        # Add to the lookup table
        #
        print(f"Lookup familyId {familyId}")
        familyId = data[0]['patient']['familyId']
        if familyId not in familyId2pid and pid.startswith('P'):
            familyId2pid[familyId] = pid


        # TODO: Add paths to fastq on BaseSpace
        #
        fastqs = get_fastq_paths_on_bs(data[0]["labAliquotId"])
        #sample_infos.append(fastqs)

        cases.append(sample_infos)



[2023-07-27@13:29:07] Connecting to https://nanuq.cqgc.hsj.rtss.qc.ca/nanuqMPS/ws/GetClinicalSampleInfoWS?name=21479
[2023-07-27@13:29:07] Got information for biosample 21479 a.k.a. GM231036
[2023-07-27@13:29:07] Getting HPO terms from Phenotips by Labeled EID CHUSJ3432166

Lookup familyId 23-04480
[2023-07-27@13:29:08] Connecting to https://nanuq.cqgc.hsj.rtss.qc.ca/nanuqMPS/ws/GetClinicalSampleInfoWS?name=21679
[2023-07-27@13:29:08] Got information for biosample 21679 a.k.a. GM231031
[2023-07-27@13:29:08] Getting HPO terms from Phenotips by Labeled EID CHUSJ3311560

Lookup familyId JACQUET
[2023-07-27@13:29:08] Connecting to https://nanuq.cqgc.hsj.rtss.qc.ca/nanuqMPS/ws/GetClinicalSampleInfoWS?name=21792
[2023-07-27@13:29:09] Got information for biosample 21792 a.k.a. 23-04480-T1
[2023-07-27@13:29:09] Getting HPO terms from Phenotips by Labeled EID CHUS1633799

Lookup familyId JACQUET
[2023-07-27@13:29:09] Connecting to https://nanuq.cqgc.hsj.rtss.qc.ca/nanuqMPS/ws/GetClinicalSampleI

In [26]:
familyId2pid

{'JACQUET': 'P0000104', '23-04480': 'P0000117'}

In [27]:
df = pd.DataFrame(cases)
df.columns = ['sample_name', 'biosample', 'relation', 'gender', 'label', 
                'mrn', 'cohort_type', 'date_of_birth(YYYY-MM-DD)', 
                'status', 'family', 'case_group_number', 'phenotypes', 'hpos']
df = df.sort_values(by=['family', 'relation'], ascending=[True, False])
df['case_group_number'] = df.apply(lambda x: familyId2pid[x['family']], axis=1)
df


,sample_name,biosample,relation,gender,label,mrn,cohort_type,date_of_birth(YYYY-MM-DD),status,family,case_group_number,phenotypes,hpos
2,23-04480-T1,21792,PROBAND,FEMALE,CHUS,1633799,TRIO,05/05/2023,AFF,23-04480,P0000117,"Large for gestational age,Polyhydramnios,Prema...","HP:0001520,HP:0001561,HP:0001622,HP:0001290,HP..."
4,23-04365-T1,21794,MTH,FEMALE,CHUS,1393563,TRIO,21/11/1997,UNK,23-04480,P0000117,,
3,23-04471-T1,21793,FTH,MALE,CHUS,423785,TRIO,14/12/1995,UNK,23-04480,P0000117,,
1,GM231031,21679,PROBAND,MALE,CHUSJ,03311560,DUO,06/07/2015,AFF,JACQUET,P0000104,"Premature birth,Maternal hypertension,Moderate...","HP:0001622,HP:0008071,HP:0025664,HP:0000708,HP..."
0,GM231036,21479,MTH,FEMALE,CHUSJ,03432166,DUO,28/12/1986,UNF,JACQUET,P0000104,,


Emedgene's scan to get full pathes from BSSH is too slow when there are lots of files, like in the case of CHUSJ. For constructing the full path to datasets, we need to fill-in something like this:

`/projects/0000000000/biosamples/2038096/datasets/ds.13f35b390f3c44f5a2c89600cfbfac87/sequenced files/114375967`

We can get information on datasets with `bs`.

`bs -c cac1 datasets list --input-biosample 21479`

    +----------+-------------------------------------+-----------------+----------------+
    |   Name   |                 Id                  |  Project.Name   | DataSetType.Id |
    +----------+-------------------------------------+-----------------+----------------+
    | 21479_L2 | ds.329b32d9c9c44334ab2d624b83786e70 | PRAGMatIQ_CHUSJ | common.fastq   |
    | 21479_L1 | ds.2c97eae5808641079896e63bbce00319 | PRAGMatIQ_CHUSJ | common.fastq   |
    +----------+-------------------------------------+-----------------+----------------+

In [30]:
from lib.gapp import BSSH
bssh = BSSH()
path = '/projects/'


In [38]:
project_id   = bssh.get_project_id('PRAGMatIQ_CHUSJ')
biosample_id = bssh.get_biosample_id(21479)
dataset_id   = bssh.get_dataset_id(biosample_id)
fastqs = bssh.get_sequenced_files(21479)
fastqs


{
  "Id": "165328238",
  "Href": "https://api.cac1.sh.basespace.illumina.com/v2/files/165328238",
  "HrefContent": "https://api.cac1.sh.basespace.illumina.com/v2/files/165328238/content",
  "Name": "21479_S1_L002_R1_001.fastq.gz",
  "ContentType": "application/x-gzip",
  "Size": 39717006901,
  "Path": "21479_S1_L002_R1_001.fastq.gz",
  "IsArchived": false,
  "DateCreated": "2023-06-11T03:01:45.0000000Z",
  "DateModified": "2023-06-11T03:01:45.0000000Z",
  "ETag": "a84bae735cb12db45da5963bfdacefad-7576",
  "IdAsLong": 165328238
}
{
  "Id": "165328239",
  "Href": "https://api.cac1.sh.basespace.illumina.com/v2/files/165328239",
  "HrefContent": "https://api.cac1.sh.basespace.illumina.com/v2/files/165328239/content",
  "Name": "21479_S1_L002_R2_001.fastq.gz",
  "ContentType": "application/x-gzip",
  "Size": 41419160608,
  "Path": "21479_S1_L002_R2_001.fastq.gz",
  "IsArchived": false,
  "DateCreated": "2023-06-11T03:01:45.0000000Z",
  "DateModified": "2023-06-11T03:01:45.0000000Z",
  "ETag

['/projects/0000000000/biosamples/21479/datasets/ds.329b32d9c9c44334ab2d624b83786e70/sequenced files/165328238',
 '/projects/0000000000/biosamples/21479/datasets/ds.329b32d9c9c44334ab2d624b83786e70/sequenced files/165328239',
 '/projects/0000000000/biosamples/21479/datasets/ds.2c97eae5808641079896e63bbce00319/sequenced files/165328171',
 '/projects/0000000000/biosamples/21479/datasets/ds.2c97eae5808641079896e63bbce00319/sequenced files/165328172']